In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import pygame
import sys
from PIL import Image
from IPython import display as ipythonDisplay
#해당 그래프를 인라인으로 표시
#%matplotlib inline

In [2]:
'''
cartpole 
state  : [위치, 속도, 각, 각속도]
action : left(-1), right(+1)
reward : sim이 끝이 나지 않는 한 계속 +1
1) pole이 수직으로부터 15도 이상 기울어질 때
2) cart가 중심으로부터 2.4 만큼 떨어질 때
episode가 끝난다.
'''
env = gym.make('CartPole-v1', render_mode='rgb_array')
# state : 시작할 때의 값(초기화)
observation = env.reset()

In [3]:
'''
reward : 이전 action에 의해서 얻은 보상의 총합 -> 항상 학습은 reward를 증가시키는 방향으로
observation : 관찰에 따른 구체적인 요소들의 array
done : episode가 끝났을 경우 true, 끝나지 않았다면 false
현 예제에서는 reward = 1 <- done == false
**done = true 이후 reward를 0 반환
'''
#100번의 time step
for _ in range(100):
    #visualize
    env.render()
    #random한 action
    action = env.action_space.sample()    
    #action을 취했을 때의 결과값
    observation, reward, done, _, info = env.step(action)    
env.close()

c:\Users\CHR\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
c:\Users\CHR\anaconda3\Lib\site-packages\gym\envs\classic_control\cartpole.py:177: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


In [4]:
#episode 2번 진행
for i_episode in range(2):
    observation = env.reset()
    for k in range(100):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, _, info = env.step(action)
        #done이 true가 됐을 경우 break
        if done :
            print("Episode finished after {} timesteps".format(k+1))
            break

env.close()

Episode finished after 28 timesteps
Episode finished after 10 timesteps


In [5]:
n_bins_pos = 10
n_bins_vel = 10
n_bins_ang = 10
n_bins_anv = 10

'''
문제상황
1. 상태는 연속적이지 않음 s0, s1, s2.....
2. 그러나 실제 문제를 해결하는 상황에서는 state가 연속적인 경우가 많음
3. 따라서 연속적인 상태를 처리하기 위해서 연속적인 state를 양자화 시켜주는 구문이 필요함
'''
# 연속적인 상태를 discrete하게 변환해주는 함수
def map_discrete_state(state):
    #state를 unpacking
    pos, vel, ang, anv = state
    #histogram : 
    '''
    NumPy의 np.histogram() 함수의 출력 형식은 두 가지 배열로 구성된 튜플(tuple)입니다:
        첫 번째 배열은 각 bin의 빈도수를 포함합니다.
        두 번째 배열은 각 bin의 경계값을 포함합니다.
    np.where() 함수는 조건을 만족하는 요소의 인덱스를 반환합니다. 
    이때, 반환되는 값은 튜플(tuple) 형태입니다. 
    튜플의 첫 번째 요소는 조건을 만족하는 요소의 행(row) 인덱스들의 배열이고, 
    두 번째 요소는 조건을 만족하는 요소의 열(column) 인덱스들의 배열입니다.
    '''
    #clip : 들어온 값이 설정된 값(-2, 2) 이상이라면 한계값으로 설정
    #bins : 들어온 값으로 range를 등분
    #이후 1이 되는 idx를 반환
    idx_pos = np.where(np.histogram(np.clip(pos,-2,2), bins = n_bins_pos, range = (-2,2))[0] == 1)[0][0]
    idx_vel = np.where(np.histogram(np.clip(vel,-2,2), bins = n_bins_vel, range = (-2,2))[0] == 1)[0][0]
    idx_ang = np.where(np.histogram(np.clip(ang,-0.4,0.4), bins = n_bins_ang, range = (-0.4,0.4))[0] == 1)[0][0]
    idx_anv = np.where(np.histogram(np.clip(anv,-2,2), bins = n_bins_anv, range = (-2,2))[0] == 1)[0][0]
    '''
     따라서 states 배열의 특정 위치에 1을 할당함으로써 해당하는 상태를 나타내는 원-핫 인코딩(one-hot encoding) 벡터를 생성합니다. 
     최종적으로 reshape(-1,1)을 사용하여 states 배열을 1차원 배열로 변형합니다.
    '''

    states = np.zeros([n_bins_pos, n_bins_vel, n_bins_ang, n_bins_anv])
    states[idx_pos, idx_vel, idx_ang, idx_anv] = 1
    #-1로 reshape되었으므로 배열의 모양이 자동으로 조정되었을 것입니다.
    states = states.reshape(-1,1)

    s = np.where(states == 1)[0][0]
    
    return s  



## Q_learning

In [6]:
alpha = 0.3
gamma = 0.9
n_states = n_bins_pos*n_bins_vel*n_bins_ang*n_bins_anv
n_actions = 2
# 0과 1 사이의 값으로 Q_table을 초기화 시켜줌
Q_table = np.random.uniform(0, 1, (n_states, n_actions))
#생성 시키는 episode의 개수
for episode in range(801):
    
    done = False
    state, _ = env.reset()
    
    count = 0
    
    while not done:
        
        count += 1        
        s = map_discrete_state(state)    

        # Exploration vs. Exploitation for action       
        epsilon = 0.1
        '''
        입실론 - 탐욕법을 활용하여 0과 1 사이를 랜덤하게 뽑은 값이
        입실론 값보다 작다면 탐험을,
        입실론 값보다 크다면 활용을 한다.
        '''
        # 탐험
        if np.random.uniform() < epsilon:
            #가능한 action 중에서 랜덤하게 하나를 뽑는다.
            a = env.action_space.sample()
        # 활용
        else:
            # action 중 Q table에서 그 값을 최대로 하는 action을 선택한다.
            a = np.argmax(Q_table[s, :])                

        # next state and reward
        next_state, reward, done, _, _ = env.step(a) 
                
        if done:        
            reward = -100        
            Q_table[s, a] = reward
                 
        else:                                               
            # Temporal Difference Update
            '''
            TD를 이용해서 smooth하게 Q_table을 update를 해줌
            '''
            next_s = map_discrete_state(next_state)
            Q_table[s, a] = (1 - alpha)*Q_table[s, a] + alpha*(reward + gamma*np.max(Q_table[next_s, :]))

        state = next_state
    if episode % 100 == 0:
        print("Episode: {} steps: {}".format(episode, count))
        
env.close()


Episode: 0 steps: 9
Episode: 100 steps: 31
Episode: 200 steps: 75
Episode: 300 steps: 113
Episode: 400 steps: 114
Episode: 500 steps: 136
Episode: 600 steps: 168
Episode: 700 steps: 162
Episode: 800 steps: 157


In [12]:

state, _ = env.reset()

done = False
cnt = 0
env.render()
screen = env.render()
images = [Image.fromarray(screen)]
while not done:
    if cnt % 10 == 0:
        screen = env.render()
        images.append(Image.fromarray(screen))
    s = map_discrete_state(state)
    a = np.argmax(Q_table[s,:])
    
    next_state, _, done, _, _ = env.step(a)      
    state = next_state  
    cnt +=1  
env.close()

image_file = 'cartpole-v1.gif'
# loop=0: loop forever, duration=1: play each frame for 1ms
images[0].save(
    image_file, save_all=True, append_images=images[1:], loop=0, duration=1)